<a href="https://colab.research.google.com/github/maurimendiluce/Clases-Mate2/blob/main/Sistema_de_ecuaciones_y_aplicaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Sistemas de ecuaciones diferenciales**

$$\left\{
\begin{align}
\textbf{x}'(t) &= \textbf{f}(t,\textbf{x}(t)) \\
\textbf{x}'(t_0) &= \textbf{x}_0
\end{align}
\right.
$$
por ejemplo, 
$$\left\{
\begin{align}
x'(t) &= f_1(t,x(t),y(t)) \\
y'(t) &= f_2(t,x(t),y(t))  \\
x(0) &= x_0, \quad \ y(0) = y_0
\end{align}
\right.
$$

Para resolver podemos usar, por ejemplo, método de Euler o el comando $\texttt{solve_ivp}$ de la libreria $\texttt{scipy.integrate}$ (https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html). Veamoslo en dos aplicaciones.

### **Ejemplo 1: Modelo depredador-presa (Lotka–Volterra)**

Cuando estudiamos el crecimiento de una población se tiene en cuenta la tasa de crecimiento de la población natural, es decir bajo la condición de recursos ilimitados. También se considera el caso de recursos que se agotan, pero siempre tomando una sola especie. Vamos a analizar ahora la evolución de poblaciones que interactúan. Por simplicidad vamos a tomar dos especies, la primera que llamaremos predador, la que se alimenta de la segunda especie, a la que denominaremos presa. Asumimos algunas hipótesis:

*   los predadores tienen como única fuente de alimentación a las presas
*   las presas cuentan con recursos ilimitados,
*   la única amenaza de las presas son los predadores.

Denotamos $x(t)$ e $y(t)$ el número de individuos de predadores y de presas a tiempo $t$, respectivamente. En ausencia de presas, $y(t) = 0$, $x(t)$ decae con tasa $\alpha$; mientras que en ausencia de predadores, $x(t) = 0$, $x(t)$ crece con tasa $\beta$. Además, los encuentros entre individuos de cada especie hacen crecer la población de los predadores, mientras que decrece la poblaci´on de las presas, en forma proporcional al número de encuentros $n(t)$ por unidad de tiempo. Siendo que $n(t)$ es proporcional al número de predadores y al número de presas, vemos que $n(t) = k x(t) y(t)$. De esto modo, se obtiene el sistema Lokta-Volterra:

$$\left\{
\begin{align}
x'(t) &= -αx(t)+γx(t)y(t) \\
y'(t) &= βy(t)-δx(t)y(t)
\end{align}
\right.
$$

donde $γ$ y $δ$ ponderan el efecto de $n(t)$ en la tasa de crecimiento de predarores y presas, respectivamente.

Tomar $α=0.25$, $β=1$, $γ=δ=0.01$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
#Ejemplo con Euler
def f1(t,x,y,alpha,beta,gamma,delta):
    return(-alpha*x + gamma*x*y)
def f2(t,x,y,alpha,beta,gamma,delta):
    return(beta*y - delta*x*y)

def euler(f1, f2, t0, tn, x0, y0, n):
    t = np.linspace(t0,tn,n+1)
    x = np.zeros(n+1)
    y = np.zeros(n+1)
    x[0] = x0
    y[0] = y0
    h = (tn-t0)/n
    for i in range(1,n+1):
        x[i] = x[i-1]+h*f1(t[i-1], x[i-1], y[i-1],alpha,beta,gamma,delta)
        y[i] = y[i-1]+h*f2(t[i-1], x[i-1], y[i-1],alpha,beta,gamma,delta)
    return((t,x,y))

t0=0
tn=60
alpha=0.25
beta=1
gamma=0.01
delta=0.01
x0=25
y0=100
n=1000

(t,x,y) = euler(f1, f2, t0, tn, x0, y0, n)
plt.plot(t, x, label="predador")
plt.plot(t, y, label="presa")
plt.legend(loc = "upper left")

In [ ]:
#grafico las trayectorias
plt.plot(x,y)

In [ ]:
#Ejemplo con solve_ivp

def lotkavolterra(t,z,alpha,beta,gamma,delta):
    x, y = z
    return [-alpha*x + gamma*x*y, beta*y - delta*x*y]

sol = solve_ivp(lotkavolterra, [t0, tn], [x0, y0], args=(alpha, beta, gamma, delta),dense_output=True)

t = np.linspace(t0, tn, 300)
z = sol.sol(t)
plt.plot(t, z.T)
plt.xlabel('t')
plt.legend(['predador', 'presa'], shadow=True)
plt.title('Lotka-Volterra System')

In [ ]:
#Problema linealizado
def g1(t,x,y,alpha,beta,gamma,delta):
    return(#completar)
def g2(t,x,y,alpha,beta,gamma,delta):
    return(#completar)

(t,xx,yy) = euler(g1, g2, t0, tn,x0, y0, n)
plt.plot(t, xx, label="predador")
plt.plot(t, yy, label="presa")
plt.legend()

In [ ]:
plt.plot(xx,yy)
plt.xlabel('predador')
plt.ylabel('presa')

## **Ejemplo 2: Modelo epidemiológico SIR**

Uno de los modelos epidemilógicos más usados es el denominado modelo SIR, que fue propuesto por W. O. Kermack y A. G. McKendrick en 1927.

En una población de tamaño fijo $N$ en la que se ha desatado una epidemia que se propaga mediante contagio, en un tiempo $t$ los individuos pueden estar en tres estados distintos:


*   susceptibles $S(t)$,
*   infectados $I(t)$,
*   recuperados $R(t)$.

($S(t) + I(t) + R(t) = N$)

Los susceptibles se pueden infectar cuando entran en contacto con contagiados, con un parámetro $β$ denominado tasa de infección, y cuyo valor depende de si la enfermedad es más o menos contagiosa. Así mismo, los infectados se recuperan con el tiempo, con un parámetro γ denominado tasa de recuperación y que depende del tiempo que suele durar la enfermedad; una vez que se recuperan, los individuos son inmunes, ya no vuelven a ser susceptibles (de hecho, también podemos pensar que no todos los individuos se «recuperan», sino que pueden morir a causa de la enfermedad: ambos tipos de casos están recogidos en $R(t)$, y ya no afectan al desarrollo de la epidemia). De acuerdo al comportamiento descrito, la evolución de la epidemia se modela mediante el siguiente sistema de ecuaciones diferenciales:

$$\left\{
\begin{align}
S'(t) &= -βS(t)I(t)/N\\
I'(t) &= βS(t)I(t)/N-γI(t)\\
R'(t) &= γI(t)
\end{align}
\right.
$$

En el instante inicial, el número de infectados es $I(0) = I_0$; además, $S(0) = N − I_0$ y $R(0) = 0$. Los parámetros $β$ y $γ$ son, respectivamente, la tasa de transmisión (mide la probabilidad de que un susceptible se infecte cuando entra en contacto con un infectado) y la tasa de recuperación (de tal manera que el periodo medio de recuperación es $1/γ$) de la enfermedad, y dependiendo de cuál sea su valor el desarrollo de la epidemia (cuánto dura o el número total de infectados, por ejemplo) puede ser muy distinto.
En una epidemia, un parámetro muy importante es $R_0 = \frac{β}{γ}$
que se denomina «tasa básica de reproducción», y que representa el número de nuevos infectados producidos por un sólo infectado si toda la población es susceptible (esta idea se ve mejor sin
más que escribir $R_0 = \frac{γ^{-1}}{β^{-1}}$, dado que $γ^{-1}$ y $β^{-1}$ son el periodo medio de recuperación y el tiempo típico entre contactos). Cuanto más pequeño sea $R_0$, de manera más lenta evolucionará la epidemia (en la práctica, y para una epidemia real concreta, la observación de la epidemia permite medir $R_0$ y, a partir de ahí, estimar $β$).

Veamos un ejemplo de evolución de una enfermedad contagiosa a lo 
largo de un año (el tiempo lo medimos en meses) sobre una población de $N = 100000$ personas, parámetros $β = 1$
y $γ = 1/4$ (la enfermedad dura, de media, cuatro meses), y un número inicial de infectados $I_0 = 2000$.

$a)$ Utilizar el método de Euler modificado para resolver el sistema. 

$b)$ Graficar las aproximaciones de $S(t)$, $I(t)$ y $R(t)$ en un mismo grafico para analizar la evolución de la epidemia.

In [14]:
N=100000
beta=1
gamma=0.25
def f(t, v):
    (S,I,R) = v
    return(np.array([#completar]))

Método: 
$$
x_{n}=x_{n-1}+hf\left(t_{n-1}+{\frac {h}{2}},x_{n-1}+{\frac {h}{2}}f(t_{n-1},x_{n-1})\right)
$$

In [ ]:
def eulerModificado(f,t0,tn,x0,n):
  t=np.linspace(t0,tn,n+1)
  m=len(x0)
  x=np.zeros((m,n+1))
  x[:,0]=x0
  h=(tn-t0)/n
  for i in range(1,n+1):
    #metodo
  return(t,x)

x0=np.array([#completar])
(t,x)=eulerModificado(f,0,12,x0,1000)

#graficar en el mismo gráfico las tres funciones